In [1]:
import functions as fxn
import numpy as np
import pandas as pd

In [2]:
columns = ['INSTNM', 'HIGHDEG', 'CONTROL', 'REGION', 'LOCALE', 'LO_INC_DEBT_N', 'MD_INC_DEBT_N', 'HI_INC_DEBT_N',
           'LOAN_EVER', 'PELL_EVER', 'PCTPELL', 'ICLEVEL', 'CURROPER', 'TUITFTE', 'CDR3', 'INEXPFTE']
sheets = fxn.read_in_CollegeScorecard(columns)
full_df = fxn.concatenate_all_sheets(sheets)
full_df_nonan = fxn.quick_clean(full_df, columns=['PCTPELL', 'CDR3'])

In [4]:
filled_full_df_nonan = fxn.back_fill_from_year(full_df_nonan, 2017)

In [9]:
filled_full_df_nonan['Public_or_Private'] = filled_full_df_nonan.CONTROL.map({2:0, 1:1, 3:1})
filled_full_df_nonans = fxn.quick_clean(filled_full_df_nonan, columns=['CDR3', 'HI_INC_DEBT_N', 'Public_or_Private'])
filled_full_df_nonans.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 37518 entries, (0, 2011-01-01 00:00:00) to (6735, 2016-01-01 00:00:00)
Data columns (total 18 columns):
INSTNM               37518 non-null object
HIGHDEG              37518 non-null int64
REGION               37518 non-null float64
TUITFTE              37454 non-null float64
INEXPFTE             37455 non-null float64
PCTPELL              37518 non-null float64
CDR3                 37518 non-null float64
LO_INC_DEBT_N        37518 non-null object
MD_INC_DEBT_N        37518 non-null object
HI_INC_DEBT_N        37518 non-null object
LOAN_EVER            37502 non-null object
PELL_EVER            37502 non-null object
ICLEVEL              37518 non-null float64
YEAR                 37518 non-null datetime64[ns]
LOCALE               0 non-null float64
CURROPER             0 non-null float64
CONTROL              37518 non-null float64
Public_or_Private    37518 non-null int64
dtypes: datetime64[ns](1), float64(9), int64(2), object(6)
memory

In [10]:
fxn.create_logistic_regression(filled_full_df_nonans, ['CDR3', 'HI_INC_DEBT_N'], 'Public_or_Private'
                              ).score(filled_full_df_nonans[['CDR3', 'HI_INC_DEBT_N']], df['Public_or_Private'])

ValueError: could not convert string to float: 'PrivacySuppressed'

In [ ]:
plt.figure(figsize=(14,8))
plt.scatter('CDR3', 'Public_or_Private', data=df)
# plt.plot(df.CDR3.sort_values(), df.Predicted_Public_or_Private.sort_values(), c='r')
plt.plot(df.CDR3.sort_values(), df.ProbCorrect_Predicted_Public_or_Private.sort_values(), c='r')
plt.xlabel('Scaled 3-year Default Rate')
plt.ylabel('College is Either Public of Private For-Profit')
plt.title('Higher Default Rates Predict Public and For-Profit Status')
plt.xlim(-0.05,0.6)
plt.show()